In [3]:
import tensorflow 

In [4]:
import pandas as pd
import numpy as np
from keras.layers import Dense,SimpleRNN,GRU ,LSTM ,Embedding
from keras.models import Sequential
from keras.utils import to_categorical

In [5]:
messages = pd.read_csv(r"D:\denis\spam.csv",encoding ='cp1252')

In [6]:
messages.shape

(6776, 5)

In [7]:
messages.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [8]:
messages= messages.iloc[:,[0 ,1]]
messages.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [9]:
messages.rename(columns = {"v1" : "label" ,"v2" :"message"},inplace = True)

In [10]:
messages.label.replace({"ham" :0 , "spam" :1},inplace =True)

In [11]:
messages_x = messages.iloc[:,1]
messages_y = messages.iloc[:,0]

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
x_train , x_test , y_train, y_test= train_test_split(messages_x , messages_y , test_size = .2)

In [14]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)    # its kind of one hot encoding

In [15]:
max_num_words =1000 ## how many word do we consider from the entire CORPUs
seq_len = 50  ##
embedding_size = 100## vector length of each word

In [16]:
y_train[1]


array([1., 0.], dtype=float32)

In [17]:
y_train.shape

(5420, 2)

In [18]:
from keras.preprocessing.text import Tokenizer

In [19]:
# glove embedding stanford university

In [20]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [21]:
tokenizer = Tokenizer(num_words = max_num_words)
tokenizer.fit_on_texts(messages.message)
x_train = tokenizer.texts_to_sequences(x_train)  # we convert tesxt tto sequences of IDs
x_train = pad_sequences(x_train, maxlen = seq_len)
x_test = tokenizer.texts_to_sequences(x_test)
x_test = pad_sequences(x_test , maxlen = seq_len)
model = Sequential()  ## initialize the network
model.add(Embedding(input_dim =max_num_words,
                     input_length = seq_len,
                      output_dim = embedding_size))

In [22]:
model.add(LSTM(5))  #  5 neiron in your layer
model.add(Dense(2 , activation = 'softmax')) # this 2 bcz u have 2 target varible
from keras.optimizers import Adam
adam = Adam(lr = .001)
model.compile(optimizer = adam , loss = 'categorical_crossentropy')

In [25]:
model.fit(x_train, y_train, epochs = 5 , batch_size = 32 , validation_split = .2)

Epoch 1/5
136/136 [==============================] - 6s 43ms/step - loss: 0.3664 - val_loss: 0.1888
Epoch 2/5
136/136 [==============================] - 4s 28ms/step - loss: 0.1320 - val_loss: 0.0999
Epoch 3/5
136/136 [==============================] - 4s 29ms/step - loss: 0.0676 - val_loss: 0.0681
Epoch 4/5
136/136 [==============================] - 4s 28ms/step - loss: 0.0421 - val_loss: 0.0579
Epoch 5/5
136/136 [==============================] - 4s 32ms/step - loss: 0.0264 - val_loss: 0.0528


In [26]:
pred1 = model.predict_classes(x_test)  # do the prediction

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [27]:
from sklearn.metrics import confusion_matrix

In [29]:
tab1 = confusion_matrix(pred1 , y_test[:,1])
print(tab1)
print("---")
print("Acc of the test data is --> " , tab1.diagonal().sum()/ tab1.sum()*100, "%")

[[1156   11]
 [   5  184]]
---
Acc of the test data is -->  98.82005899705014 %
